In [2]:
import numpy as np
import igl
import meshplot as mp

In [3]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [4]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})


    

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [10]:
def find_closed_point(point, points):
    distance=np.linalg.norm(points-point,axis=1)
    return np.argmin(distance)


eps=igl.bounding_box_diagonal(pi)*0.01
piplus=np.zeros_like(pi)
piminus=np.zeros_like(pi)
for (i,point) in enumerate(pi):
    temp=point+eps*ni[i]
    neweps=eps
    while find_closed_point(temp,pi)!=i:
        neweps=neweps/2
        temp=point+neweps*ni[i]
    piplus[i]=temp
for (i,point) in enumerate(pi):
    temp=point-eps*ni[i]
    neweps=eps
    while find_closed_point(temp,pi)!=i:
        neweps=neweps/2
        temp=point-neweps*ni[i]
    piminus[i]=temp


p=mp.plot(pi,c=np.ones_like(pi)*np.array([0, 0, 1]), shading={"point_size": 8})
p.add_points(piplus,c=np.ones_like(pi)*np.array([1, 0, 0]), shading={"point_size": 8})
p.add_points(piminus,c=np.ones_like(pi)*np.array([0, 1, 0]), shading={"point_size": 8})

bv,bf=igl.bounding_box(pi)
print(bv)
print(bf)
tet_grid(pi,np.min(np.array(bv),axis=0),np.max(np.array(bv),axis=0))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

[[ 32.7  16.7   5.8]
 [ 32.7  16.7 -99. ]
 [ 32.7 -63.7   5.8]
 [ 32.7 -63.7 -99. ]
 [-22.7  16.7   5.8]
 [-22.7  16.7 -99. ]
 [-22.7 -63.7   5.8]
 [-22.7 -63.7 -99. ]]
[[2 0 6]
 [0 4 6]
 [5 4 0]
 [5 0 1]
 [6 4 5]
 [5 7 6]
 [3 0 2]
 [1 0 3]
 [3 2 6]
 [6 7 3]
 [5 1 3]
 [3 7 5]]


C:\Users\m\AppData\Local\Temp\ipykernel_30416\3916701390.py:13: RuntimeWarning: divide by zero encountered in divide
  deltay = delta[1]/(ny-1)


TypeError: only integer scalar arrays can be converted to a scalar index

# MLS function

In [6]:
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 10

In [7]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function
center = np.array([0., 0., 0.])
radius = 1
fx = np.linalg.norm(x-center, axis=1) - radius

In [8]:
# Treshold fx to visualize inside outside

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Marching to extract surface

In [ ]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})